In [1]:
import numpy as np
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime
import scipy
import json
import random
from math import floor

from covid19model.optimization import objective_fcns
from covid19model.models import models
from covid19model.models.utils import draw_sample_COVID19_SEIRD
from covid19model.models.time_dependant_parameter_fncs import ramp_fun, google_lockdown
from covid19model.data import mobility, sciensano, model_parameters
from covid19model.visualization.output import population_status, infected, _apply_tick_locator
from covid19model.visualization.optimization import plot_fit, traceplot

plt.rcParams.update({'font.size': 12, 'axes.grid':True, 'grid.linewidth':0.3})
pd.plotting.register_matplotlib_converters()

# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
%load_ext snakeviz

In [3]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(dataset='willem_2012', spatial='arr')

In [4]:
# Need to initiate I also with the co-infection model, otherwise divison by zero occurs
initial_states = {'S': initN, 'E': np.ones(initN.shape), 'I': np.ones(initN.shape)}

In [5]:
params = model_parameters.get_COVID19_SEIRD_parameters(spatial='arr')
dummy_spatial_model = models.COVID19_SEIRD_spatial(initial_states, params, spatial='arr')


In [6]:
start_calibration = '2021-01-01'
#end_calibration = '2020-11-12'
end_sim = '2021-09-01'

In [ ]:
#dummy_spatial_model.sim(time=end_sim, start_date=start_calibration)

In [7]:
pi = params['pi']
place = params['place']
Nc = params['Nc']
G = place.shape[0] # spatial stratification
N = Nc.shape[0] # age stratification
area = params['area']
xi = params['xi']
beta = params['beta']
K = params['K']
s = params['s']

In [8]:
T = np.ones([G,N])
A = np.ones([G,N])
I = np.ones([G,N])
S = initial_states['S']
V = np.ones([G,N]) 
alpha = np.ones([G,N]) 

# Loop 1

In [9]:
#%%timeit
T_eff = np.zeros([G,N]) # initialise
A_eff = np.zeros([G,N])
I_eff = np.zeros([G,N])
for g in range(G):
    for i in range(N):
        sumT = 0
        sumA = 0
        sumI = 0
        sumAlpha = 0
        # patch h is taken, so iterator for a sum is gg
        for gg in range(G):
            term1 = (1 - pi[i]) * np.identity(G)[gg][g]
            term2 = pi[i] * place[gg][g]
            sumT += (term1 + term2) * T[gg][i]
            sumA += (term1 + term2) * A[gg][i]
            sumI += (term1 + term2) * I[gg][i]
        T_eff[g][i] = sumT
        A_eff[g][i] = sumA
        I_eff[g][i] = sumI


In [10]:
%%timeit
result_T = (1-pi)*np.matmul(np.identity(G),T) + pi*np.matmul(np.transpose(place),T)
result_A = (1-pi)*np.matmul(np.identity(G),A) + pi*np.matmul(np.transpose(place),A)
result_I = (1-pi)*np.matmul(np.identity(G),I) + pi*np.matmul(np.transpose(place),I)

113 µs ± 6.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
133000/93

In [ ]:
result_A.round(9)==A_eff.round(9)

# Loop 2

In [ ]:
# np.outer gebruiken om extra dimensie te maken??

In [384]:
%%timeit
Susc = np.zeros([G,G,N])
V_Susc = np.zeros([G,G,N]) # vaccinated people that did not reach immunity
for gg in range(G):
    for hh in range(G):
        for i in range(N):
            Susc[gg][hh][i] = pi[i] * place[gg][hh] * S[gg][i] + (1 - pi[i]) * np.identity(G)[gg][hh] * S[gg][i]       
            V_Susc[gg][hh][i] = pi[i] * place[gg][hh] * V[gg][i] + (1 - pi[i]) * np.identity(G)[gg][hh] * V[gg][i] 

272 ms ± 9.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [389]:
%%timeit
Susc = np.zeros([G,G,N])
#V_Susc = np.zeros([G,G,N])
for i in range(N):
    Susc[:,:,i] =  pi[i]*place*S[:,[i]] + (1 - pi[i])*np.identity(G)*S[:,[i]]
#    V_Susc[:,:,i] =  pi[i]*place*V[:,[i]] + (1 - pi[i])*np.identity(G)*V[:,[i]]


537 µs ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [381]:
#Susc.round(6)==Susc2.round(6)

In [386]:
%%timeit
Susc3 = (pi[np.newaxis, np.newaxis,:]*place[:,:,np.newaxis]*S[:,np.newaxis,:] + 
         (1-pi[np.newaxis, np.newaxis,:])*np.identity(G)[:,:,np.newaxis]*S[:,np.newaxis,:])

239 µs ± 3.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [387]:
272000/239

1138.0753138075313

In [366]:
### This is what happens under the hood in previous cell

# place_3D = np.repeat(place[:, :, np.newaxis], 9, axis=2)
# S_3D = np.repeat(S[:, np.newaxis, :], 43, axis=1)
# pi_2D = np.repeat(pi[np.newaxis, :], 43, axis=0)
# pi_3D = np.repeat(pi_2D[np.newaxis, :, :], 43, axis=0)
# Susc_3D = pi_3D * place_3D * S_3D

In [383]:
#Susc.round(6)==Susc3.round(6)

# Loop 3

In [12]:
T_eff = (1-pi)*np.matmul(np.identity(G),T) + pi*np.matmul(np.transpose(place),T)


In [13]:
# Density dependence per patch: f[patch]
T_eff_total = T_eff.sum(axis=1)
rho = T_eff_total / area
f = 1 + (1 - np.exp(-xi * rho))

In [18]:
#%%timeit
Ti = T.sum(axis=0)
denom = np.zeros(N)
for gg in range(G):
    value = f[gg] * T_eff[gg]
    denom += value
zi = Ti / denom

In [19]:
#%%timeit
Ti = T.sum(axis=0)
zi1 = Ti / np.matmul(np.transpose(T_eff),f)

In [ ]:
162/10

In [20]:
zi1.round(8) == zi1.round(8)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

# Loop 4

In [21]:
alpha_eff = (1-pi)*np.matmul(np.identity(G),alpha) + pi*np.matmul(np.transpose(place),alpha)

In [29]:
#%%timeit
B = np.zeros([G,N])
for gg in range(G):
    for i in range(N):
        sumj = 0
        for j in range(N):
            beta_weighted_av = (1-alpha_eff[gg,j])*beta + alpha_eff[gg,j]*K*beta
            term = beta_weighted_av * s[i] * zi[i] * f[gg] * Nc[i,j] * (I_eff[gg,j] + A_eff[gg,j]) / T_eff[gg,j]
            #term = beta * s[i] * Nc[i,j] * (I_eff[gg,j] + A_eff[gg,j]) / T_eff[gg,j]
            sumj += term
        B[gg][i] = sumj

In [30]:
#%%timeit
beta_weighted_av = (1-alpha_eff)*beta + alpha_eff*K*beta
multip = np.outer(f, s*zi)*(I_eff + A_eff) / T_eff
B2 = beta_weighted_av*np.matmul(multip, np.transpose(Nc))

In [27]:
21000/28

750.0

In [31]:
B.round(6)==B2.round(6)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True, 

# Loop 5

In [86]:
%%timeit
# Infection from sum over all patches
dS_inf = np.zeros([G,N])
dV_inf = np.zeros([G,N])
for gg in range(G):
    for i in range(N):
        sumhh = 0
        sumhh_V = 0
        for hh in range(G):
            term = Susc[gg][hh][i] * B[hh][i]
            sumhh += term
            term_V = V_Susc[gg][hh][i] * B[hh][i]
            sumhh_V += term_V                    
        dS_inf[gg][i] = sumhh
        dV_inf[gg][i] = sumhh_V

44 ms ± 3.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [87]:
%%timeit
dS_inf2 = (Susc*B).sum(axis=1)
dV_inf2 = (V_Susc*B).sum(axis=1)

141 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [88]:
44000/141

312.0567375886525

In [84]:
dS_inf==dS_inf2

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True, 

In [85]:
dV_inf==dV_inf2

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True, 